# 🚀 Hierarchical Subchat System - Kaggle GPU Testing

## 📋 Setup Checklist (Do Once):

### 1️⃣ **Add Kaggle Secrets** (Most Important!)
Go to: **https://www.kaggle.com/settings** → Add-ons → Secrets

Add these two secrets:
- **`GROQ_API_KEY`** = Your Groq API key (for query decomposition)
- **`GITHUB_TOKEN`** = Your GitHub personal access token (for pushing results)

### 2️⃣ **Enable Internet in This Notebook**
- Click "⚙️ Settings" (top right)
- Turn ON **"Internet"** toggle
- Click "Save"

### 3️⃣ **Make Sure This Notebook is PRIVATE**
- Never share secrets in public notebooks!

---

## ▶️ Run Order:
1. Run all cells in order (the notebook will automatically handle everything)
2. The vLLM model (Qwen-3 14B) will load on Kaggle's GPUs
3. Your repo will be cloned from GitHub
4. Tests will run and results will be pushed back to GitHub
5. Pull the results on your local machine to analyze

---

## 🎯 What This Notebook Does:
- ✅ Uses Kaggle's free GPU to run large language models
- ✅ Tests your hierarchical subchat architecture with real models
- ✅ Generates performance logs for buffer size analysis
- ✅ Automatically syncs results back to your GitHub repo

In [19]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GITHUB_TOKEN")
secret_value_1 = user_secrets.get_secret("GROQ_API_KEY")
secret_value_2 = user_secrets.get_secret("HuggingFACEHUB_access_token")
secret_value_3 = user_secrets.get_secret("LANGCHAIN_API_KEY")

# ✅ IMPORTANT: Set them in os.environ so other code can access them
os.environ["GITHUB_TOKEN"] = secret_value_0
os.environ["GROQ_API_KEY"] = secret_value_1
os.environ["HuggingFACEHUB_access_token"] = secret_value_2
os.environ["LANGCHAIN_API_KEY"] = secret_value_3
os.environ["LLM_BACKEND"] = "vllm"

# Print the tokens (first 4 and last 4 characters for security)
print("="*60)
print("🔐 SECRETS LOADED AND SET IN ENVIRONMENT")
print("="*60)
print(f"✅ GITHUB_TOKEN: {secret_value_0[:4]}...{secret_value_0[-4:]}")
print(f"✅ GROQ_API_KEY: {secret_value_1[:4]}...{secret_value_1[-4:]}")
print(f"✅ HuggingFACEHUB_access_token: {secret_value_2[:4]}...{secret_value_2[-4:]}")
print(f"✅ LANGCHAIN_API_KEY: {secret_value_3[:4]}...{secret_value_3[-4:]}")
print(f"✅ LLM_BACKEND: vllm")
print("="*60)

🔐 SECRETS LOADED AND SET IN ENVIRONMENT
✅ GITHUB_TOKEN: gith...tWfg
✅ GROQ_API_KEY: gsk_...l6gr
✅ HuggingFACEHUB_access_token: hf_E...GaQC
✅ LANGCHAIN_API_KEY: lsv2...ea2f
✅ LLM_BACKEND: vllm


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# CHECKING OUT MY GPU WORKINGW

*** repo: https://github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM ***


In [8]:
! uv pip uninstall -q --system 'tensorflow'
! uv pip install -q --system  'vllm' 'triton' 'logits-processor-zoo' 'numpy<2'

In [9]:
import os
import re
import logging
from pathlib import Path
import pickle
import json
import joblib
import shutil
import glob
from tqdm.auto import tqdm
import warnings

import numpy as np
import pandas as pd



# For Qwen
import torch
import vllm
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor


AttributeError: module 'torch._subclasses.fake_tensor' has no attribute 'UnsupportedMutationAliasingException'

In [ ]:
# vLLM V1 does not currently accept logits processor so we need to disable it
# https://docs.vllm.ai/en/latest/getting_started/v1_user_guide.html#deprecated-features
os.environ["VLLM_USE_V1"] = "0"

#model_path = "/kaggle/input/qwen2.5-coder/transformers/32b-instruct-awq/1"
model_path = "/kaggle/input/qwen-3/transformers/14b-awq/1"
llm = vllm.LLM(
    model_path,
    quantization='awq',
    tensor_parallel_size=torch.cuda.device_count(),
    gpu_memory_utilization=0.91,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True,
    enable_prefix_caching=True
)
tokenizer = llm.get_tokenizer()

In [ ]:
from vllm import SamplingParams

def stream_generate(llm, prompt):
    sampling_params = SamplingParams(
        temperature=0.2,
        top_p=0.9,
        max_tokens=512,
    )

    for output in llm.generate(
        [prompt],
        sampling_params,
        
    ):
        yield output.outputs[0].text


# ── Usage ────────────────────────────────────────────────
prompt = """You are a helpful assistant.
User: Explain tensor parallelism in simple terms.
Assistant:"""

for token in stream_generate(llm, prompt):
    print(token, end="", flush=True)


In [ ]:
prompt = """what is quantum computing?"""

for token in stream_generate(llm, prompt):
    print(token, end="", flush=True)

In [ ]:
print('h')

h


# test github

In [21]:
# Load secrets from Kaggle's secure environment
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

print("="*60)
print("🔐 LOADING SECRETS FROM KAGGLE")
print("="*60)

# Try to load GROQ_API_KEY
try:
    GROQ_API_KEY = user_secrets.get_secret("GROQ_API_KEY")
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY
    print("✅ GROQ_API_KEY loaded successfully")
    print(f"   Key length: {len(GROQ_API_KEY)} characters")
except Exception as e:
    print(f"⚠️  GROQ_API_KEY not found: {e}")
    print("   Add it in Kaggle Settings → Secrets")
    GROQ_API_KEY = None

# Try to load GITHUB_TOKEN
try:
    GITHUB_TOKEN = user_secrets.get_secret("GITHUB_TOKEN")
    os.environ["GITHUB_TOKEN"] = GITHUB_TOKEN
    print("✅ GITHUB_TOKEN loaded successfully")
    print(f"   Token length: {len(GITHUB_TOKEN)} characters")
except Exception as e:
    print(f"⚠️  GITHUB_TOKEN not found: {e}")
    print("   Add it in Kaggle Settings → Secrets")
    GITHUB_TOKEN = None

# Set LLM backend to use vLLM (local model on Kaggle GPU)
os.environ["LLM_BACKEND"] = "vllm"  # We'll use the vLLM model loaded above
print("\n✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)")

print("="*60)

🔐 LOADING SECRETS FROM KAGGLE
✅ GROQ_API_KEY loaded successfully
   Key length: 56 characters
✅ GITHUB_TOKEN loaded successfully
   Token length: 93 characters

✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)
✅ GITHUB_TOKEN loaded successfully
   Token length: 93 characters

✅ LLM_BACKEND set to 'vllm' (using Kaggle GPU)


In [22]:
# Check GPU availability and configuration
import torch

print("="*60)
print("🔍 ENVIRONMENT CHECK")
print("="*60)
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
print(f"✅ CUDA version: {torch.version.cuda}")
print(f"✅ Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"\n🎮 GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"   Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")

print(f"\n✅ Current working directory: {os.getcwd()}")
print("="*60)

🔍 ENVIRONMENT CHECK
✅ PyTorch version: 2.6.0+cu124
✅ CUDA available: False
✅ CUDA version: 12.4
✅ Number of GPUs: 0

✅ Current working directory: /kaggle/working


In [26]:
# Clone the kaggle-run branch from GitHub (PUBLIC READ - no auth needed)
import subprocess

REPO_URL = "https://github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM.git"
REPO_DIR = "Subchat-Trees"
BRANCH = "kaggle-run"

print("="*60)
print("📥 CLONING REPOSITORY")
print("="*60)

# Remove existing directory if present
if os.path.exists(REPO_DIR):
    print(f"⚠️  Removing existing {REPO_DIR} directory...")
    shutil.rmtree(REPO_DIR)

# Clone the specific branch (no authentication needed for public repos)
# Skip LFS files to avoid bandwidth quota issues
print(f"🔄 Cloning {BRANCH} branch (skipping LFS files)...")
print("   No authentication required for cloning (public repo)")

# Set environment variable to skip LFS files
clone_env = os.environ.copy()
clone_env["GIT_LFS_SKIP_SMUDGE"] = "1"

result = subprocess.run(
    ["git", "clone", "-b", BRANCH, "--single-branch", REPO_URL, REPO_DIR],
    capture_output=True,
    text=True,
    env=clone_env
)

if result.returncode == 0:
    print(f"✅ Successfully cloned {BRANCH} branch!")
    print(f"📂 Repository location: {os.path.abspath(REPO_DIR)}")
    
    # List key directories to verify
    print("\n📁 Key directories found:")
    key_dirs = ["backend", "backend/src", "backend/dataset"]
    for dir_path in key_dirs:
        full_path = os.path.join(REPO_DIR, dir_path)
        if os.path.exists(full_path):
            print(f"   ✅ {dir_path}")
        else:
            print(f"   ❌ {dir_path} (not found)")
else:
    print(f"❌ Clone failed: {result.stderr}")
    
print("="*60)

📥 CLONING REPOSITORY
⚠️  Removing existing Subchat-Trees directory...
🔄 Cloning kaggle-run branch (skipping LFS files)...
   No authentication required for cloning (public repo)
✅ Successfully cloned kaggle-run branch!
📂 Repository location: /kaggle/working/Subchat-Trees

📁 Key directories found:
   ✅ backend
   ✅ backend/src
   ✅ backend/dataset
✅ Successfully cloned kaggle-run branch!
📂 Repository location: /kaggle/working/Subchat-Trees

📁 Key directories found:
   ✅ backend
   ✅ backend/src
   ✅ backend/dataset


In [27]:
# Configure git identity
os.chdir(REPO_DIR)

print("="*60)
print("⚙️  CONFIGURING GIT")
print("="*60)

!git config user.name "moonmehedi"
!git config user.email "the.mehedi.hasan.moon@gmail.com"

print("✅ Git identity configured!")
print(f"   User: moonmehedi")
print(f"   Email: the.mehedi.hasan.moon@gmail.com")

# Verify current branch
branch_result = subprocess.run(["git", "branch", "--show-current"], capture_output=True, text=True)
print(f"\n✅ Current branch: {branch_result.stdout.strip()}")
print("="*60)

os.chdir("..")  # Return to parent directory

⚙️  CONFIGURING GIT
✅ Git identity configured!
   User: moonmehedi
   Email: the.mehedi.hasan.moon@gmail.com

✅ Current branch: kaggle-run
✅ Git identity configured!
   User: moonmehedi
   Email: the.mehedi.hasan.moon@gmail.com

✅ Current branch: kaggle-run


In [28]:
# Test git push by creating a test log file
os.chdir(REPO_DIR)

print("="*60)
print("🧪 TESTING GIT PUSH CAPABILITY")
print("="*60)

# Create logs directory if it doesn't exist
os.makedirs("kaggle_logs", exist_ok=True)

# Create a detailed test log file
test_log_file = "kaggle_logs/connection_test.log"
from datetime import datetime
current_time = datetime.now()

with open(test_log_file, "w") as f:
    f.write("="*60 + "\n")
    f.write("🔬 KAGGLE GPU TEST RUN - CONNECTION VERIFICATION\n")
    f.write("="*60 + "\n\n")
    
    f.write(f"📅 Test Date: {current_time.strftime('%Y-%m-%d')}\n")
    f.write(f"⏰ Test Time: {current_time.strftime('%H:%M:%S UTC')}\n")
    f.write(f"📍 Timestamp: {pd.Timestamp.now()}\n\n")
    
    f.write("="*60 + "\n")
    f.write("🎮 GPU CONFIGURATION\n")
    f.write("="*60 + "\n")
    f.write(f"GPU Count: {torch.cuda.device_count()}\n")
    
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            f.write(f"\nGPU {i}:\n")
            f.write(f"  - Name: {torch.cuda.get_device_name(i)}\n")
            f.write(f"  - Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB\n")
    else:
        f.write("⚠️  No GPU detected\n")
    
    f.write("\n" + "="*60 + "\n")
    f.write("📊 ENVIRONMENT INFO\n")
    f.write("="*60 + "\n")
    f.write(f"PyTorch Version: {torch.__version__}\n")
    f.write(f"CUDA Available: {torch.cuda.is_available()}\n")
    f.write(f"Working Directory: {os.getcwd()}\n")
    
    f.write("\n" + "="*60 + "\n")
    f.write("✅ TEST STATUS: SUCCESS\n")
    f.write("="*60 + "\n")
    f.write(f"\nThis log was generated from Kaggle notebook\n")
    f.write(f"Push attempt at: {current_time.isoformat()}\n")

print(f"✅ Created detailed test log: {test_log_file}")

# Add and commit
!git add kaggle_logs/connection_test.log
!git commit -m "Test: Kaggle GPU verification - {current_time.strftime('%Y-%m-%d %H:%M:%S')}"

# Set up remote URL with token
if "GITHUB_TOKEN" in os.environ:
    repo_url_with_token = f"https://{os.environ['GITHUB_TOKEN']}@github.com/moonmehedi/Subchat-Trees-A-Scalable-Architecture-for-Multi-Threaded-Dialogue-and-Context-Isolation-in-LLM.git"
    !git remote set-url origin {repo_url_with_token}
    print("\n🔄 Pushing to GitHub...")
    push_result = subprocess.run(["git", "push", "origin", BRANCH], capture_output=True, text=True)
    
    if push_result.returncode == 0:
        print("✅ Successfully pushed to GitHub!")
        print(f"   📁 Check: kaggle_logs/connection_test.log")
        print(f"   📅 Pushed at: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print("   💡 Pull on your local machine to verify sync")
    else:
        print(f"❌ Push failed: {push_result.stderr}")
else:
    print("⚠️  Cannot push: GitHub token not configured")

print("="*60)

os.chdir("..")  # Return to parent directory

🧪 TESTING GIT PUSH CAPABILITY
✅ Created detailed test log: kaggle_logs/connection_test.log
[kaggle-run 7a97502] Test: Kaggle GPU verification - 2025-12-13 16:32:42
 1 file changed, 27 insertions(+)
 create mode 100644 kaggle_logs/connection_test.log
[kaggle-run 7a97502] Test: Kaggle GPU verification - 2025-12-13 16:32:42
 1 file changed, 27 insertions(+)
 create mode 100644 kaggle_logs/connection_test.log

🔄 Pushing to GitHub...

🔄 Pushing to GitHub...
✅ Successfully pushed to GitHub!
   📁 Check: kaggle_logs/connection_test.log
   📅 Pushed at: 2025-12-13 16:32:42
   💡 Pull on your local machine to verify sync
✅ Successfully pushed to GitHub!
   📁 Check: kaggle_logs/connection_test.log
   📅 Pushed at: 2025-12-13 16:32:42
   💡 Pull on your local machine to verify sync


# testing ends